In [ ]:
!pip install spacy
!pip install textacy
!python -m spacy download en_core_web_sm
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━

# Initial Exploration

In [ ]:
# attempting to do rel extraction on HF datasets
import textacy

In [ ]:
import spacy,en_core_web_sm
from spacy.matcher import Matcher

nlp = en_core_web_sm.load()
matcher = Matcher(nlp.vocab)

# figure out some pos tags
adv = nlp("I am in the house")
for token in adv:
    print(token.pos_)

PRON
AUX
ADP
DET
NOUN


In [ ]:
sentence = 'The author is writing a new book.'
doc = nlp(sentence)

# inspect the doc first
for token in doc:
  print (token.pos_)

print ('='*40)
pattern = [{'POS': {'IN': ['VERB', 'AUX']}, 'OP': '?'}, {'POS': 'ADV', 'OP': '*'}, {'POS': 'VERB', 'OP': '+'}] # r'<VERB>?<ADV>*<VERB>+'
matcher.add("VERB_PHRASE", [pattern])
matches = matcher(doc)

for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span.text)

DET
NOUN
AUX
VERB
DET
ADJ
NOUN
PUNCT
is writing
writing


In [ ]:
# expand the pattern to incorporate a proper noun before and after
# note: these patterns would not match things like "J.R.R. Tolkien is the author of The Hobbit" (noun in the verb phrase)
sentence = "Reneé Rapp acts in Mean Girls"
doc_2 = nlp(sentence)

full_pattern = [{'POS': 'PROPN', 'OP': '+'}] + pattern + [{'POS': 'ADP', 'OP': '?'}, {'POS': 'PROPN', 'OP': '+'}]
matcher.add("BASIC_RELATION", [full_pattern])
matches = matcher(doc_2)

for match_id, start, end in matches:
    matched_span = doc_2[start:end]
    print(matched_span.text)

Rapp acts in Mean
Reneé Rapp acts in Mean


Based on this initial analysis, we need a few additional features to achieve sufficiently high-quality matches. Firstly, we need to match on entities, instead of tokens. "Mean Girls" is one proper noun entity, but two tokens. Secondly, we should use statistical models to identify subject-object relations.